In [7]:
import pandas as pd 
import os
import json
from datetime import date, datetime
import update_survey

In [18]:
access_token = os.environ.get('access_token_surveymonkey')
survey_id = 315864260

update_survey.update_survey_data(survey_id=survey_id, access_token=access_token)

with open('data.json', 'r') as data_file:
    json_data = json.load(data_file)

with open('details.json') as details_file: 
    json_details = json.load(details_file)

def parse_timestamp(text):
    d = text.replace('+00:00', '')
    return datetime.strptime(d, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d')

# Map Question ID to Question Text
id = pd.json_normalize(json_details['pages'], ['questions'])['id']
question_text = pd.json_normalize(json_details['pages'], ['questions', 'headings'])['heading']
questions = dict(zip(id, question_text))

# Map ID to college variable
id = []
college_value = []
for i in range(len(json_data['data'])):
    id.append(json_data['data'][i]['id'])
    college_value.append(json_data['data'][i]['custom_variables']['College'])

colleges = dict(zip(id, college_value))

# Map ID to semester variable
semester_value = []
for i in range (len(json_data['data'])):
    semester_value.append(json_data['data'][i]['custom_variables']['Semester'])

semesters = dict(zip(id, semester_value))

# Map ID to program variable
program_value = []
for i in range(len(json_data['data'])):
    program_value.append(json_data['data'][i]['custom_variables']['Program'])

programs = dict(zip(id, program_value))

# Map ID to module variable
module_value = []
for i in range (len(json_data['data'])):
    module_value.append(json_data['data'][i]['custom_variables']['Module'])

modules = dict(zip(id, module_value))

# Building the dataframe
json_df = pd.json_normalize(json_data['data'], ['pages', 'questions', 'answers'], ['id', ['questions', 'answers', 'id']])
json_df['college'] = json_df['id'].map(colleges)
json_df['semester'] = json_df['id'].map(semesters)
json_df['program'] = json_df['id'].map(programs)
json_df['module'] = json_df['id'].map(modules)
json_df['question'] = json_df['questions.answers.id'].map(questions)

# Rename columns
json_df.rename(columns={
    "choice_metadata.weight" : "answer",
    "questions.answers.id" : "question_id",
    "id" : "respondent_id"
}, inplace=True)

# Reshaping
json_df = json_df[[
    'respondent_id',
    'college',
    'semester',
    'program',
    'module',
    'question_id',
    'question',
    'answer',
    'text'
]]

json_df['answer'].fillna(json_df['text'])
json_df

# # Questions as headers - values / index / column headers
# df = json_df.pivot_table(['answer'], ['respondent_id', 'college', 'program', 'semester', 'module'], 'question')
# df.to_excel('response_data.xlsx')
# df


,respondent_id,college,semester,program,module,question_id,question,answer,text
0,13270572817,BPC,202103,Audio,MPP01,742623547,The lecturer(s) provided a positive learning e...,2,NaN
1,13270572817,BPC,202103,Audio,MPP01,742623636,The workload was appropriate for this module,3,NaN
2,13270572817,BPC,202103,Audio,MPP01,742623741,The expectations of the assessments were made ...,3,NaN
3,13270572817,BPC,202103,Audio,MPP01,742623924,Learning resources were easily available for t...,3,NaN
4,13270572817,BPC,202103,Audio,MPP01,742624050,The lecturer(s) was reasonably available for c...,2,NaN
...,...,...,...,...,...,...,...,...,...
74,13270727929,TPC,202202,Web,MPP03,742686813,Overall I am satisfied with the quality of my ...,5,NaN
75,13270727929,TPC,202202,Web,MPP03,742686920,Overall I am satisfied with the quality of my ...,6,NaN
76,13270727929,TPC,202202,Web,MPP03,742687252,What did you like most about this trimester?,NaN,fgfgfgf
77,13270727929,TPC,202202,Web,MPP03,742687310,What would you change about this trimester?,NaN,fgfgfg
